In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pulp as pl
import plotly.express as px
import plotly.graph_objects as go
import utils as util
import os


season = 'summer'
priorityScores = [500, 400, 300, 200]
min_bucket = 1
winter = 'RMS-MultiKPI-Input-KisDonemi-GercekVeri.xlsx'
summer = 'RMS-MultiKPI-Input-YazDonemi-GercekVeri.xlsx'

data = pd.ExcelFile(eval(season))
tasks = pd.read_excel(data, sheet_name='Tasks')

resources = pd.read_excel(data, sheet_name='Resources')
priorityContent = pd.read_excel(data, sheet_name='PriorityContents')

tasks['StartDateTime'] = pd.to_datetime(tasks['StartDate'].astype(str) + ' ' + tasks['StartTime'].astype(str)) 
tasks['EndDateTime'] = pd.to_datetime(tasks['EndDate'].astype(str) + ' ' + tasks['EndTime'].astype(str))

tasks['EndWithBuffer'] = tasks['EndDateTime'] + pd.to_timedelta(tasks['BufferTime-min'], unit='minutes')

In [2]:
taskHeatmap = util.heatmap(tasks)

In [3]:
compatibilities = util.compute_compatabilities(tasks, resources)

priorities = util.read_prios(tasks, resources, priorityContent, priorityScores)

U = util.utilities(data)

taskList = tasks.TaskId
resourceList = resources.ResourceId

In [4]:
problem = pl.LpProblem(f'AdnanMenders_{season}Data', pl.LpMaximize)

x = {}


###DECISION VARIABLES
for t in taskList:
    for r in resourceList:
        x[t, r] = pl.LpVariable('x_%s,%s' % (t, r), lowBound=0, upBound=compatibilities.loc[t, r], cat=pl.LpBinary)

U_max = pl.LpVariable('U_max', lowBound=0, upBound=1, cat=pl.LpContinuous)
U_min = pl.LpVariable('U_min', lowBound=0, upBound=1, cat=pl.LpContinuous)
###OBJECTIVE FUNCTION
problem += pl.lpSum([(U.loc[var] + np.sum([priorities[i].loc[var] for i in range(len(priorities))])) * x[var] for var in x])



In [5]:
#CONSTRAINTS

#i) assign one task to one resource
for t in taskList:
    problem += pl.lpSum([x[t, r] for r in resourceList]) <= 1

In [6]:
#ii) assign one resource to one task at a time
for idx, row in taskHeatmap.iterrows():
    tasks_in_time_step = set(dict(row[row==1]).keys())
    for r in resourceList:
        cons = [x[t, r] for t in tasks_in_time_step if (t, r) in x]
        if len(cons) > 1:
            constraint_for_time_bucket = pl.lpSum(cons) <= 1
            problem += constraint_for_time_bucket




In [9]:
### RESOURCE UTILIZATION
# a) find resource with max and min utilization
totalTime = (tasks['EndDateTime'].max() - tasks['StartDateTime'].min()).total_seconds() / 60
for r in resourceList:
    rUtil = (pl.lpSum(util.resource_utilization(r, tasks, x)) / totalTime)
    problem += rUtil - U_max <= 0
    problem += rUtil - U_min >= 0

problem += U_max - U_min <= 0.5

#for r in resourceList:
#    rUtil = (pl.lpSum(util.resource_utilization(r, tasks, x)) / totalTime)
#    problem += rUtil <= 0.9
#    problem += rUtil >= 0.2


-U_max + 0.019868205915742895*x_1,29 + 0.006193331021098739*x_10,29 + 0.005549224594875399*x_100,29 + 0.0036664519645361357*x_1000,29 + 0.0028737055938396505*x_1001,29 + 0.0032700787789283338*x_1002,29 + 0.0031214388344551875*x_1003,29 + 0.003616905316205381*x_1004,29 + 0.010602982708000605*x_1005,29 + 0.002477332408231848*x_1006,29 + 0.012089382153251185*x_1007,29 + 0.003963731853482428*x_1008,29 + 0.00832383689205354*x_1009,29 + 0.004558291631375013*x_101,29 + 0.007729277114160955*x_1010,29 + 0.0031709854827859426*x_1011,29 + 0.016548580488483806*x_1012,29 + 0.003864638557340037*x_1013,29 + 0.0031214388349743065*x_1014,29 + 0.003914185205151673*x_1015,29 + 0.0031709854827859426*x_1016,29 + 0.0036664519645361357*x_1017,29 + 0.00946340980054619*x_1018,29 + 0.004062825150143938*x_1019,29 + 0.0036169053167245*x_102,29 + 0.0030223455383127963*x_1020,29 + 0.004013278501813183*x_1021,29 + 0.004508744983563377*x_1022,29 + 0.0064906109100450315*x_1023,29 + 0.0028737055938396505*x_1024,29 + 0.

In [8]:
solver = pl.getSolver('GUROBI_CMD')
solver.solve(problem)

Set parameter Username
Set parameter LogFile to value "gurobi.log"
Using license file /home/memo/gurobi.lic
Academic license - for non-commercial use only - expires 2024-12-04

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Pop!_OS 22.04 LTS")
Copyright (c) 2023, Gurobi Optimization, LLC



KeyboardInterrupt: 

In [ ]:
assignments = pd.DataFrame(columns=['TaskId', 'ResourceId'])
for var in x:
    #print(x[var].varValue)
    if x[var].varValue == 1:
        
        #new_row = pd.DataFrame({'TaskId': var[0], 'ResourceId':var[1]})
        assignments.loc[len(assignments)] = [var[0], var[1]]


print(assignments)
assignments.to_csv(f'{season}_assignments.csv')
##print gantt chart
gantt_df = pd.DataFrame({'ResourceId': assignments.ResourceId.astype(str), 'TaskId': assignments.TaskId,
                         'StartDateTime': tasks.loc[assignments.TaskId - 1].StartDateTime,
                         'EndDateTime': tasks.loc[assignments.TaskId - 1].EndDateTime})

fig = px.timeline(gantt_df, x_start='StartDateTime', x_end='EndDateTime', y='ResourceId', color='TaskId', color_continuous_scale='rainbow')
fig.write_html(f'{season}_gantt.html')
fig.show()